## Preprocessament

In [1]:
import pandas as pd
import numpy as np

Lectura de dades

In [2]:
activitats_v = pd.read_csv("data/activitats_valencia.csv")
activitats_a = pd.read_csv("data/activitats_alacant.csv")
activitats_c = pd.read_csv("data/activitats_castello.csv")
ajuntaments_info = pd.read_csv("data/ajuntaments_info_raw.csv")

* Seleccio de variables i canvi de nom

In [3]:
columns = ["Marca temporal", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
columns_v = ["Marca de temps", "Localitat", "Data", "Categoria", "En cas d'altres, quina?", "Nom de l'activitat ", "Companyia, artista", "Lloc", "Llengua de l'activitat", "Preu", "Regidoria organitzadora", "Llengua de la publicitat", "Font",  "Persona que ha introduït les dades"]
column_rename = ["marca_temps", "localitat", "data", "categoria", "categoria_alternativa", "nom_activitat", "companyia", "lloc", "llengua_activitat", "preu", "regidoria", "llengua_publicitat", "font", "persona_dades"]
columns_info_rename = ["ajuntament", "provincia", "comarca", "n_habitants", "index_envelliment_2021", "pob_turistica", "govern_22_23", "subv_us_valencia_2022", "predomini_ling", "regidoria_cultura"]

In [4]:
activitats_v = activitats_v[columns_v]
activitats_v.columns = column_rename
activitats_c = activitats_c[columns]
activitats_c.columns = column_rename
activitats_a = activitats_a[columns]
activitats_a.columns = column_rename
ajuntaments_info.columns = columns_info_rename

* Juntem els 3 datasets per a conformar el dataset activitats

In [5]:
activitats = pd.concat([activitats_a, activitats_v, activitats_c], axis = 0)

In [6]:
activitats.shape

(7378, 14)

## Preprocessament de dades

* Tranformacions en variables

**ajuntaments_info**

pob_turistica

In [7]:
# Adegim el valor "No" a aquelles poblacions que falten dades
ajuntaments_info["pob_turistica"].loc[ajuntaments_info["pob_turistica"].isnull()] = "No"

c:\Users\quimm\Anaconda3\envs\venv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


index_envelliment_2021

In [8]:
ajuntaments_info["index_envelliment_2021"] = pd.to_numeric(ajuntaments_info["index_envelliment_2021"].str.replace(",", ".") )

subv_us_valencia_2022

In [9]:
ajuntaments_info["subv_us_valencia_2022"] = pd.to_numeric(ajuntaments_info["subv_us_valencia_2022"].str.replace(".", "").str.replace("€", "").str.replace(",", ".") )

C:\Users\quimm\AppData\Local\Temp/ipykernel_20744/2853699601.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ajuntaments_info["subv_us_valencia_2022"] = pd.to_numeric(ajuntaments_info["subv_us_valencia_2022"].str.replace(".", "").str.replace("€", "").str.replace(",", ".") )


**activitats**

llengua_activitat

In [10]:
def assign_value_more_than_2_languages(llengua_activitat):
    """
    si la variable té dos llengues o més, el valor canvia a "Dos o més llengües"
    """
    llengua_activitat_split = llengua_activitat.split(", ")

    if len(llengua_activitat_split) > 1:
        if "Activitat sense llengua" not in llengua_activitat_split and "No hi ha informació" not in llengua_activitat_split:
            return "Dos o més llengües"
    
    return llengua_activitat

In [11]:
activitats["llengua_activitat"] = activitats["llengua_activitat"].apply(lambda x: assign_value_more_than_2_languages(x))

localitat

In [12]:
activitats = activitats.loc[activitats["localitat"] != "Bilingüe"]
activitats = activitats.loc[activitats["localitat"] != "Monforte del Cid"]

data

In [13]:
activitats = activitats.loc[~pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month.isin([4,5])]

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 233-02-22 00:00:00

Variables de text

In [ ]:
activitats.nom_activitat=activitats.nom_activitat.str.replace(",", ";")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\n", "")
activitats.nom_activitat = activitats.nom_activitat.str.replace("\r", "")
activitats.companyia = activitats.companyia.str.replace(",", ";")
activitats.lloc = activitats.lloc.str.replace(",", ";")
activitats.preu = activitats.preu.str.replace(",", ";")

* Creació de noves variables

Preu_mitjà

In [ ]:
activitats["preu_mitja"] = activitats.preu.str.findall(r"\d+[,'.]?\d*")
activitats["preu_mitja"].loc[activitats.preu.notnull()] = activitats.loc[activitats.preu.notnull()].preu_mitja.apply(lambda x: np.float64(np.mean([float(num.replace("'", ".").replace(",", ".")) for num in x])))

c:\Users\quimm\Anaconda3\envs\venv\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
type(activitats["preu_mitja"].loc[activitats.preu.notnull()][0])

pandas.core.series.Series

In [ ]:
# Modifiquem el . per , per problemes de codificacio en els analistes
#activitats["preu_mitja"] = activitats["preu_mitja"].apply(lambda num: str(num).replace(".", ","))

Categoria New

In [ ]:
def assign_category(category_alternative):

    if type(category_alternative) != str:
        category_new = np.nan
    else:
        category_alternative = category_alternative.lower()
        if "fall" in category_alternative:
            category_new = "Falles"
        elif "màg" in category_alternative:
            category_new = "Màgia"
        elif "visita" in category_alternative:
            category_new = "Activitats sobre patrimoni"
        elif "circ" in category_alternative:
            category_new = "Circ"
        elif "concu" in category_alternative:
            category_new = "Concurs"
        elif "premi" in category_alternative:
            category_new = "Premis"
        elif "taller" in category_alternative:
            category_new = "Taller"
        elif "espectac" in category_alternative:
            category_new = "Espectacle"
        else:
            category_new = np.nan

    return category_new


In [ ]:
activitats["categoria_new"] = activitats["categoria_alternativa"].apply(assign_category)
activitats["categoria_new"] = np.where(activitats['categoria_new'].isnull(), activitats['categoria'], activitats['categoria_new'])

Mes

In [ ]:
activitats["Mes"] = pd.to_datetime(activitats.data, format= "%d/%m/%Y").dt.month_name()

* Nou conjunt de dades: activitats_summaries.csv

llengua activitat

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(3).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_activitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top2 = grup1.groupby("localitat").apply(lambda x : x.head(2).tail(1).reset_index(drop = True))["llengua_activitat"]
llengua_activitat_top3 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

llengua publicitat

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_publicitat"])["llengua_publicitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).head(2).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

llengua_publicitat_top1 = grup1.groupby("localitat").apply(lambda x : x.head(1).reset_index(drop = True))["llengua_activitat"]
llengua_publicitat_top2 = grup1.groupby("localitat").apply(lambda x : x.tail(1).reset_index(drop = True))["llengua_activitat"]

Language position

In [ ]:
grup1 = activitats.groupby(["localitat","llengua_activitat"])["llengua_activitat"].count()
grup1 = pd.DataFrame(grup1)
grup1.columns = ["count_llengua"]
grup1= grup1.reset_index()

grup1 = grup1.groupby("localitat").apply(lambda x : x.sort_values(by = 'count_llengua', ascending = False).reset_index(drop = True))
grup1.columns = ["localitat_col", 'llengua_activitat', 'count_llengua']

In [ ]:
pos_val = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Valencià/català"])
pos_val = pd.DataFrame(pos_val)

pos_cas = grup1.groupby("localitat").apply(lambda x : x.index[x["llengua_activitat"]=="Espanyol"])
pos_cas = pd.DataFrame(pos_cas)

In [ ]:
def get_position(array):
    "get the position of the language in the most used ones. If not used, number 5 assigned"
    try:
        return array[0][0][1]
    except:
        return 5

In [ ]:
positions_val = []

for i in range(pos_val.shape[0]):
    positions_val.append(get_position(pos_val.iloc[i]))

In [ ]:
positions_cas = []

for i in range(pos_cas.shape[0]):
    positions_cas.append(get_position(pos_cas.iloc[i]))

N activitats

In [ ]:
grup1 = activitats.groupby(["localitat"])["llengua_activitat"].count()
n_activivitats = grup1.values

* Creating datasets

activitats_summaries

In [ ]:
activitats_summaries = pd.DataFrame(llengua_activitat_top1)
activitats_summaries = activitats_summaries.reset_index()
del activitats_summaries["level_1"]
activitats_summaries.columns = ['localitat', 'llengua_activitat_top1']

In [ ]:
activitats_summaries['llengua_activitat_top2'] = llengua_activitat_top2.values
activitats_summaries['llengua_activitat_top3'] = llengua_activitat_top3.values
activitats_summaries['llengua_publicitat_top1'] = llengua_publicitat_top1.values
activitats_summaries['llengua_publicitat_top2'] = llengua_publicitat_top2.values
activitats_summaries["position_val_in_llengua_activitat"] = positions_val
activitats_summaries["position_esp_in_llengua_activitat"] = positions_cas
activitats_summaries["n_activitats"] = n_activivitats

In [ ]:
#ratio_activitats_nhabitants
aux = activitats_summaries.merge(ajuntaments_info[["ajuntament", "n_habitants"]], how ='left', left_on="localitat", right_on = "ajuntament")
activitats_summaries["ratio_activitats_nhabitants"] = aux["n_activitats"]/(aux["n_habitants"]/1000)

Saving Datasets

In [ ]:
activitats.to_csv("data/activitats.csv", index=False)

In [ ]:
activitats_summaries.to_csv("data/activitats_summaries.csv", index=False)

In [ ]:
ajuntaments_info.to_csv("data/ajuntaments_info.csv", index=False)